In [ ]:
import ssl
import urllib.request
import json
import pandas as pd

In [ ]:
#list of jobs search terms
jobs_titles=["data analyst", "media producer", "software engineer", "data scientist", "product manager", "marketing manager", "sales manager", "customer service", "accountant", "financial analyst", "human resources","Designer", "Civil Engineer", "UI Developer", "Project Manager", "Backend Engineer", "Customer Success", "DevOps Engineer", "Technician", "Test Automation", "Electrical Design", "Frontend Developer", "General Laborer", "Budget Analyst", "Financial Analyst", "Sales Development", "SLED Systems Engineer", "QA Engineer", "Technical Support", "Supply Chain Coordinator"]

#list of companies
with open("../../data/companies_names_linkedin.txt",'r',encoding='utf-8') as f:
    companies = f.readlines()
companies = companies[:500]
companies = [company.strip() for company in companies]

# build the search terms
search_terms=[]
for job in jobs_titles:
    for company in companies:
        search_terms.append(f'"{job}" "{company}" linkedin')
print(len(search_terms))

In [ ]:
def parse_search_terms(search_term):
    """
    This function gets the search term and returns the search term as a string
    :param search_term: 
    :return: the search term as a string for the google search api
    """
    search_term = search_term.replace(' ', '+')
    search_term = search_term.replace('"', '%22')
    return search_term

def get_search_results(search_term): 
    """
    This function gets the search results from Google and returns them as a json
    :param search_term: a string of the search term
    :return: a json of first page of search results from Google
    """
    # TODO: currently only return the first page of search results, its possible to get more pages (but since that costs money, and the alpha is based on the number of companies, we don't necessarily want that)
    print(f"searching for: {search_term}")
    search_term_url = parse_search_terms(search_term)
    ssl._create_default_https_context = ssl._create_unverified_context
    try:
        opener = urllib.request.build_opener(
            urllib.request.ProxyHandler(
                {'http': 'http://{username}}:{password}@{host}',
                'https': 'http://{username}}:{password}@{host}'}))
        json_result = json.loads(opener.open(f'https://www.google.com/search?q={search_term_url}&gl=af&brd_json=1').read())
    except Exception as e:
        print(f"Error: {e}")
        print(f"error for search term: {search_term}")
        json_result = None
    return json_result

In [ ]:
def parse_linkdin_name(link_text):
    """
    This function gets the link text of a person and returns the name of the person
    :param link_text: 
    :return: 
    """
    name = link_text.split("/in/")[1].split("/")[0]
    # split by -
    name = name.split("-")
    # if any part of the name is a number, remove the whole part
    name = [part for part in name if not any(char.isdigit() for char in part)]
    # join the parts
    name = " ".join(name)
    return name

In [ ]:
results={}
#read progress so far
with open("../../data/employees_extracted.json", "r",encoding='utf-8') as f:
    json_data = json.load(f)
# save it into the results dictionary
results = json_data
print(f"{len(results.keys())} search terms have already been saved")

# remove the search terms that have already been searched
search_terms = [search_term for search_term in search_terms if search_term not in results.keys()]
print(f"searching for {len(search_terms)} search terms")

# # search for the remaining search terms
# for search_term in search_terms:
#     json_result = get_search_results(search_term)
#     if json_result:
#         results[search_term] = json_result
#     else:
#         print(f"Error for search term: {search_term}")
#         print("saving the results so far and skipping")
#         print("progress so far: ", len(results.keys()))
#         with open("../../data/employees_extracted.json", "w") as f:
#             json.dump(results, f)
# print("done! saving the results")
# with open("../../data/employees_extracted.json", "w") as f:
#             json.dump(results, f)

data=pd.DataFrame()
name_from_link = []
name_from_title=[]
job_title = []
company = []

for search_term in results.keys():
    if "organic" not in results[search_term].keys():
        continue
    for result in results[search_term]["organic"]:
        link_text = result["link"]
        if "/in/" in link_text:
            name_from_link.append(parse_linkdin_name(link_text))
            name_from_title.append(result["title"].split(" -")[0])
            job_title.append(search_term.split('"')[1])
            company.append(search_term.split('"')[3])
data["name_from_link"] = name_from_link
data["name_from_title"] = name_from_title
data["job_title"] = job_title
data["company"] = company
data.to_csv("../../data/employees_extracted.csv", index=False)

In [ ]:
with open(r"C:\Users\loonn\Desktop\technion\semster 7\DataCollectionLab1\code\delete\employees_extracted3.json", "r", encoding='utf-8') as f:
    results = json.load(f)
for result in results.keys():
    # each entry is a json 
    # print(results[result].keys())
    print(results[result]["organic"])
    
data=pd.DataFrame()
#cols for the data frame
name_from_link = []
name_from_title=[]
job_title = []
company = []

for search_term in results.keys():
    for result in results[search_term]["organic"]:
        link_text=result["link"]
        if "/in/" in link_text:
            name_from_link.append(parse_linkdin_name(link_text))
            name_from_title.append(result["title"].split(" -")[0])
            job_title.append(search_term.split('"')[1])
            company.append(search_term.split('"')[3])
data["name_from_link"] = name_from_link
data["name_from_title"] = name_from_title
data["job_title"] = job_title
data["company"] = company
data.to_csv("../../data/employees_extracted.csv", index=False)

 pet example of the json data

In [ ]:
print(results.keys())
# save the json data
with open("../../data/employees_extracted.json", "w") as f:
    json.dump(results, f)

In [ ]:
#read the json data
with open("../../data/employees_extracted.json", "r") as f:
    json_data = json.load(f)
# save it into the results dictionary
results = json_data

In [ ]:
search_term = '"data analyst" "SimilarWeb" linkedin'
json_data_analyst_similarweb = get_search_results(search_term)

In [ ]:
name_from_link = []
name_from_title=[]
job_title = []
company = []
data=pd.DataFrame()
for result in json_data_analyst_similarweb["organic"]:
    link_text=result["link"]
    if "/in/" in link_text:
        name_from_link.append(parse_linkdin_name(link_text))
        name_from_title.append(result["title"].split(" -")[0])
        job_title.append(search_term.split('"')[1])
        company.append(search_term.split('"')[3])
data["name_from_link"] = name_from_link
data["name_from_title"] = name_from_title
data["job_title"] = job_title
data["company"] = company
data.to_csv("../../data/employees_extracted.csv", index=False)

In [ ]:
print(json_data_analyst_similarweb["organic"])